In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy 
import os
# import seaborn as sns
# import scipy.stats as stats
import unittest

In [2]:
df_battery = pd.read_csv('Battery_Dataset.csv')

In [3]:
import numpy as np;
import csv
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold 
import math

import sklearn
import statistics as stcs
import matplotlib.pyplot as plt
import scipy.stats as stats
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler,scale,MinMaxScaler
from scipy.stats import uniform,expon
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error

In [4]:
#One hot encoding (alphebetical order)
ohe = OneHotEncoder()
ACE = df_battery.loc[:,['anode','cathode','electrolyte']]
ACE = ohe.fit_transform(ACE)
ACE = pd.DataFrame(ACE.toarray())

ACE_col_name = ['A1','C1','C2','C3','E1','E2','E3']
for i in range(len(ACE.columns)):
    ACE = ACE.rename({ACE.columns[i]: ACE_col_name[i]}, axis=1) 
    
df_battery = pd.concat([ACE, df_battery],axis=1)

In [5]:
data = df_battery

np.random.seed(66)
def data_split (data, test_ratio):
    
    total_row = df_battery.shape[0]
    test_row = round(total_row *test_ratio)
    train_row = total_row - test_row
    
    indices =np.random.permutation(total_row)
    train_indx, test_idx =indices[:train_row], indices[train_row:]
    train,test = df_battery.iloc[train_indx,:], df_battery.iloc[test_idx,:]

    X_test = test[['A1','C1','C2','C3','E1','E2','E3','Cycle','temperature','discharge_crate']]
#    y_test = test[['Charge_Capacity (Ah)']]
#    y_test = test[['Discharge_Capacity (Ah)']]
#    y_test = test[['Charge_Energy (Wh)']]
    y_test = test[['Discharge_Energy (Wh)']]
#    y_test = test[['Coulombic_Efficiency (%)']]
#    y_test = test[['Energy_Efficiency (%)']]
    
    X_train = train[['A1','C1','C2','C3','E1','E2','E3','Cycle','temperature','discharge_crate']]
#    y_train = train[['Charge_Capacity (Ah)']]
#    y_train = train[['Discharge_Capacity (Ah)']]
#    y_train = train[['Charge_Energy (Wh)']]
    y_train = train[['Discharge_Energy (Wh)']]
#    y_train = train[['Coulombic_Efficiency (%)']]
#    y_train = train[['Energy_Efficiency (%)']]

    return train,test,X_train,y_train,X_test,y_test
    
train,test,X_train,y_train,X_test,y_test = data_split(data,0.2)


In [6]:
#k-fold cross validation
kf = KFold(n_splits=10,random_state =66,shuffle= True)
#print(X_test,y_test,X_train,y_train)

In [7]:
clf = StandardScaler(with_mean=True,with_std=True)
print(clf.fit(X_test))
print(clf.mean_)

X_test = pd.DataFrame(clf.transform(X_test))

print(clf.fit(X_train))
print(clf.mean_)
X_train = pd.DataFrame(clf.transform(X_train))

StandardScaler()
[  1.         0.296875   0.34375    0.359375   0.296875   0.359375
   0.34375  292.1875    25.3125     1.4375  ]
StandardScaler()
[1.00000000e+00 3.94531250e-01 2.65625000e-01 3.39843750e-01
 3.94531250e-01 3.39843750e-01 2.65625000e-01 2.70703125e+02
 2.53125000e+01 1.45703125e+00]


In [8]:
#Random tuning
rand_para = {'C':range(291,411),'gamma':np.logspace(-2,-1),'kernel':['rbf']}
svm_model =SVR()
n_iteration = 200
Scoring = ['explained_variance','neg_mean_squared_error',"r2"]
random_search = RandomizedSearchCV(svm_model,param_distributions=rand_para,cv =5,scoring=Scoring,
                                   error_score=0,random_state=66,return_train_score=True,n_jobs=-1,n_iter=n_iteration,verbose=10,refit=False)

random_search.fit(X_train,y_train)

#y_predict=random_search.predict(X_test)
#math.sqrt(stcs.mean((y_predict-y_test)**2))

Fitting 5 folds for each of 200 candidates, totalling 1000 fits
[CV 4/5; 1/200] START C=327, gamma=0.09540954763499938, kernel=rbf..............
[CV 4/5; 1/200] END C=327, gamma=0.09540954763499938, kernel=rbf; explained_variance: (train=0.700, test=0.801) neg_mean_squared_error: (train=-2.651, test=-1.945) r2: (train=0.698, test=0.800) total time=   0.1s
[CV 5/5; 3/200] START C=400, gamma=0.07543120063354615, kernel=rbf..............
[CV 5/5; 3/200] END C=400, gamma=0.07543120063354615, kernel=rbf; explained_variance: (train=0.734, test=0.701) neg_mean_squared_error: (train=-2.379, test=-2.781) r2: (train=0.733, test=0.697) total time=   0.2s
[CV 1/5; 5/200] START C=301, gamma=0.0517947467923121, kernel=rbf...............
[CV 1/5; 5/200] END C=301, gamma=0.0517947467923121, kernel=rbf; explained_variance: (train=0.740, test=0.669) neg_mean_squared_error: (train=-2.445, test=-2.686) r2: (train=0.737, test=0.650) total time=   0.1s
[CV 5/5; 6/200] START C=335, gamma=0.07906043210907697,

[CV 1/5; 1/200] START C=327, gamma=0.09540954763499938, kernel=rbf..............
[CV 1/5; 1/200] END C=327, gamma=0.09540954763499938, kernel=rbf; explained_variance: (train=0.750, test=0.647) neg_mean_squared_error: (train=-2.333, test=-2.937) r2: (train=0.749, test=0.618) total time=   0.1s
[CV 2/5; 3/200] START C=400, gamma=0.07543120063354615, kernel=rbf..............
[CV 2/5; 3/200] END C=400, gamma=0.07543120063354615, kernel=rbf; explained_variance: (train=0.717, test=0.626) neg_mean_squared_error: (train=-2.589, test=-3.502) r2: (train=0.716, test=0.586) total time=   0.2s
[CV 3/5; 4/200] START C=400, gamma=0.010985411419875584, kernel=rbf.............
[CV 3/5; 4/200] END C=400, gamma=0.010985411419875584, kernel=rbf; explained_variance: (train=0.692, test=0.667) neg_mean_squared_error: (train=-2.787, test=-3.374) r2: (train=0.683, test=0.653) total time=   0.1s
[CV 2/5; 6/200] START C=335, gamma=0.07906043210907697, kernel=rbf..............
[CV 2/5; 6/200] END C=335, gamma=0.0

/home/kli625/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/kli625/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/kli625/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/kli625/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change t

[CV 2/5; 1/200] START C=327, gamma=0.09540954763499938, kernel=rbf..............
[CV 2/5; 1/200] END C=327, gamma=0.09540954763499938, kernel=rbf; explained_variance: (train=0.717, test=0.625) neg_mean_squared_error: (train=-2.594, test=-3.547) r2: (train=0.715, test=0.581) total time=   0.1s
[CV 1/5; 3/200] START C=400, gamma=0.07543120063354615, kernel=rbf..............
[CV 1/5; 3/200] END C=400, gamma=0.07543120063354615, kernel=rbf; explained_variance: (train=0.750, test=0.645) neg_mean_squared_error: (train=-2.336, test=-2.946) r2: (train=0.749, test=0.617) total time=   0.1s
[CV 4/5; 4/200] START C=400, gamma=0.010985411419875584, kernel=rbf.............
[CV 4/5; 4/200] END C=400, gamma=0.010985411419875584, kernel=rbf; explained_variance: (train=0.656, test=0.774) neg_mean_squared_error: (train=-3.158, test=-2.263) r2: (train=0.640, test=0.768) total time=   0.1s
[CV 1/5; 6/200] START C=335, gamma=0.07906043210907697, kernel=rbf..............
[CV 1/5; 6/200] END C=335, gamma=0.0

/home/kli625/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/kli625/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/kli625/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/kli625/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change t

...

[CV 2/5; 136/200] END C=370, gamma=0.012648552168552958, kernel=rbf; explained_variance: (train=0.676, test=0.528) neg_mean_squared_error: (train=-3.137, test=-4.700) r2: (train=0.656, test=0.444) total time=   0.1s
[CV 4/5; 137/200] START C=365, gamma=0.019306977288832496, kernel=rbf...........
[CV 4/5; 137/200] END C=365, gamma=0.019306977288832496, kernel=rbf; explained_variance: (train=0.690, test=0.790) neg_mean_squared_error: (train=-2.780, test=-2.078) r2: (train=0.683, test=0.787) total time=   0.1s
[CV 5/5; 138/200] START C=338, gamma=0.015998587196060583, kernel=rbf...........
[CV 5/5; 138/200] END C=338, gamma=0.015998587196060583, kernel=rbf; explained_variance: (train=0.693, test=0.719) neg_mean_squared_error: (train=-2.841, test=-2.781) r2: (train=0.681, test=0.697) total time=   0.1s
[CV 3/5; 140/200] START C=328, gamma=0.06866488450042998, kernel=rbf............
[CV 3/5; 140/200] END C=328, gamma=0.06866488450042998, kernel=rbf; explained_variance: (train=0.748, test=0.

/home/kli625/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/kli625/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/kli625/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/kli625/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change t

/home/kli625/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/kli625/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/kli625/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/kli625/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change t

/home/kli625/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/kli625/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/kli625/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/kli625/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change t

RandomizedSearchCV(cv=5, error_score=0, estimator=SVR(), n_iter=200, n_jobs=-1,
                   param_distributions={'C': range(291, 411),
                                        'gamma': array([0.01      , 0.01048113, 0.01098541, 0.01151395, 0.01206793,
       0.01264855, 0.01325711, 0.01389495, 0.01456348, 0.01526418,
       0.01599859, 0.01676833, 0.01757511, 0.0184207 , 0.01930698,
       0.0202359 , 0.02120951, 0.02222996, 0.02329952, 0.02442053,
       0.025595...
       0.04094915, 0.04291934, 0.04498433, 0.04714866, 0.04941713,
       0.05179475, 0.05428675, 0.05689866, 0.05963623, 0.06250552,
       0.06551286, 0.06866488, 0.07196857, 0.0754312 , 0.07906043,
       0.08286428, 0.08685114, 0.09102982, 0.09540955, 0.1       ]),
                                        'kernel': ['rbf']},
                   random_state=66, refit=False, return_train_score=True,
                   scoring=['explained_variance', 'neg_mean_squared_error',
                            'r2'],
       

In [9]:
np.random.seed(66)

svm_model_train =SVR(kernel ='rbf',gamma=0.03393221771895328,C=387)

train_results =[]
train_results_name =['Experimental','Predicted','RMSE']


for train_index, test_index in kf.split(X_train):
    #print("Train:", train_index, "Validation:", test_index)
    X_training, X_validate = X_train.iloc[train_index], X_train.iloc[test_index]
    y_training, y_validate = y_train.iloc[train_index], y_train.iloc[test_index]
    
    np.random.seed(66)
    svm_model_train.fit(X_training,y_training)
    y_train_predicted = svm_model_train.predict(X_validate)
#    print(y_train_predicted.shape)
#     number_ob =X_validate.shape[0]
#     squared_deviations=(y_train_predicted.reshape(-1,1)-y_validate)**2
#     RMSE=math.sqrt(squared_deviations.sum()/number_ob)
    #RMSE = math.sqrt(stcs.mean((y_train_predicted-y_validate)**2))
    mse = mean_squared_error(y_train_predicted, y_validate)
    RMSE = np.sqrt(mse)
    train_results.append([y_validate,y_train_predicted,RMSE])

/home/kli625/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/kli625/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/kli625/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/kli625/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change t

In [10]:
Train_results = pd.DataFrame (train_results,columns=train_results_name)

predict_results = pd.DataFrame (Train_results.iloc[:,1])

np.average(Train_results['RMSE'])

1.6772528191031568

In [11]:
svm_model_optimal =SVR(kernel ='rbf',gamma=0.03393221771895328,C=387)
np.random.seed(66)
svm_model_optimal.fit(X_train,y_train)
y_predict=svm_model_optimal.predict(X_test)

mse = mean_squared_error(y_predict, y_test)
RMSE = np.sqrt(mse)
RMSE

/home/kli625/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


1.8033028225190049

In [12]:
#math.sqrt(stcs.mean((y_predict-y_test)**2))

#SS_res = sum((y_predict-y_test)**2)
#SS_tot = sum((y_test-stcs.mean(y_test))**2)

#1-SS_res/SS_tot

In [13]:
Train_results =pd.DataFrame (train_results,columns=train_results_name)

In [14]:
np.average(Train_results['RMSE'])

1.6772528191031568

In [15]:
predict_results = pd.DataFrame (Train_results.iloc[:,1])

#predict_results.drop(labels=None)